<a href="https://colab.research.google.com/github/osaldealwis/Philosophy-Quote-Matcher/blob/main/Philosophy_Quote_Matcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("datastax/philosopher-quotes")

# Check the dataset structure
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/574 [00:00<?, ?B/s]

philosopher-quotes.csv:   0%|          | 0.00/67.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/450 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['author', 'quote', 'tags'],
        num_rows: 450
    })
})


In [ ]:
quotes = []
authors = []

# As the dataset is under the 'train' split
for item in dataset['train']:
    # Check for non-empty quote
    if item['quote']:
        quotes.append(item['quote'])
        authors.append(item['author'])

print(f"Loaded {len(quotes)} quotes.")


Loaded 450 quotes.


In [ ]:
from sentence_transformers import SentenceTransformer

# Load a Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for all quotes
quote_embeddings = model.encode(quotes, convert_to_tensor=True)
print("Computed embeddings for all quotes.")


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Computed embeddings for all quotes.


In [ ]:
import torch
from torch.nn.functional import cosine_similarity

def get_similar_quote(user_text, top_k=1):
    # Encode the user input
    user_embedding = model.encode(user_text, convert_to_tensor=True)

    # Compute cosine similarities between the user embedding and all quote embeddings
    similarities = cosine_similarity(user_embedding, quote_embeddings)

    # Get the top k indices with highest similarity
    top_results = torch.topk(similarities, k=top_k)

    results = []
    for score, idx in zip(top_results[0], top_results[1]):
        results.append({
            "quote": quotes[idx],
            "author": authors[idx],
            "similarity": score.item()
        })

    return results

# Test with a sample user input
user_input = "I feel lost and overwhelmed, like nothing makes sense."
results = get_similar_quote(user_input, top_k=3)

for res in results:
    print(f"Quote: {res['quote']}\nAuthor: {res['author']}\nSimilarity: {res['similarity']:.4f}\n")


Quote: Everything has been figured out, except how to live.
Author: sartre
Similarity: 0.4536

Quote: Enjoy life. There's plenty of time to be dead. Be kind, for everyone you meet is fighting a harder battle.
Author: plato
Similarity: 0.3759

Quote: The news that reaches your consciousness is incomplete and often not to be relied on.... Turn your eyes inward, look into your own depths, learn first to know yourself!
Author: freud
Similarity: 0.3669



In [ ]:
def main():
    while True:
        user_text = input("Enter your thoughts (or type 'exit' to quit): ")
        if user_text.lower() == 'exit':
            break

        results = get_similar_quote(user_text, top_k=1)
        best = results[0]
        print("\nHere is a quote that resonates with your thoughts:")
        print(f"\"{best['quote']}\"")
        print(f" - {best['author']}\n")

# Run the interactive loop
main()


Enter your thoughts (or type 'exit' to quit): I often wonder if our perceptions truly capture reality, or if they are merely shadows of a deeper, hidden truth.

Here is a quote that resonates with your thoughts:
"Sometimes people don't want to hear the truth because they don't want their illusions destroyed."
 - nietzsche

Enter your thoughts (or type 'exit' to quit): In the midst of chaos, I believe that true power lies in mastering our own minds rather than being controlled by external events.

Here is a quote that resonates with your thoughts:
"The measure of a man is what he does with power."
 - plato

Enter your thoughts (or type 'exit' to quit): I question whether it is our ability to choose our path that defines us, or if fate has already written our destiny.

Here is a quote that resonates with your thoughts:
"Nothing in the universe is contingent, but all things are conditioned to exist and operate in a particular manner by the necessity of the divine nature."
 - spinoza

Ente